# Extract Data #

After completing some inital exploration of the data returned from the Reed API, I will now extract the data required and store it in a pandas dataframe. 

Import libaries

In [34]:
import requests
import json
import os.path
import csv
import pandas as pd
from datetime import datetime

seach_jobs function: makes a call to the Reed API and returns the results as a list of dictionaries

In [35]:
def search_jobs(keywords, locationName, distance_in_miles):


    api_key = "440e3704-d856-440c-b07f-00ec507efd09"
    username = api_key
    password = ""

    url = f"https://www.reed.co.uk/api/1.0/search?keywords={keywords}&locationName={locationName}&distanceFromLocation={distance_in_miles}"

    response = requests.get(url,
                        auth=requests.auth.HTTPBasicAuth(
                            username, password))

    json_data = json.loads(response.text)
    return json_data['results']

List of uk cities to be used as search criteria when requesting data from the API

In [36]:
uk_cities = [
 'London',
 'Birmingham',
 'Manchester',
 'Glasgow',
 'Leeds',
 'Liverpool',
 'Bristol',
 'Sheffield',
 'Edinburgh',
 'Leicester',
 'Coventry',
 'Nottingham',
 'Hull',
 'Stoke-on-Trent',
 'Derby',
 'Southampton',
 'Reading',
 'Luton',
 'Wolverhampton',
 'Belfast',
 'Cambridge',
 'Brighton',
 'Oxford',
 'Newcastle',
 'Cardiff',
 'York']

len(uk_cities)

26

List of tech job titles to also be used as search criteria 

In [37]:
tech_job_titles = [
    "Data Scientist",
    "Data Analyst",
    "Data Engineer",
    "Cloud Engineer",
    "Software Engineer",
    "Software Developer",
    "Frontend Developer",
    "Backend Developer",
    "Full Stack Developer",
    "DevOps Engineer",
    "Machine Learning Engineer",
    "AI Engineer",
    "Mobile App Developer",
    "Game Developer",
]

I will now loop through both of the lists sending multiple requests to the API and storing all of the results in a pandas dataframe.

In [38]:
extracted_data = []

for job_title in tech_job_titles[:]:
    for city in uk_cities[:]:
        jobs_per_city = search_jobs(job_title, city, "10")
        for job in jobs_per_city:
            job['city'] = city
        extracted_data = extracted_data + jobs_per_city

len(extracted_data)

9997

We can see that the for loop has generated 10044 rows of data. I will now look at the data as I suspect some of the searches will return dupicate results.

I will first take a look at the format of the data return it should be a list of dictionaries 

In [39]:
extracted_data

[{'jobId': 54047135,
  'employerId': 409522,
  'employerName': 'WTW',
  'employerProfileId': None,
  'employerProfileName': None,
  'jobTitle': 'Senior Data Scientist',
  'locationName': 'London',
  'minimumSalary': None,
  'maximumSalary': None,
  'currency': None,
  'expirationDate': '30/12/2024',
  'date': '18/11/2024',
  'jobDescription': 'We are looking for a Data Scientist, with experience in the UK general insurance markets in either commercial or personal lines, to join Willis Towers Watson in our Data Science team which is part of our consulting practice to help us develop our Data Science advisory capability. You will work beside some of the market’s top thought people designing and implementing cutting-edge solutions to challenges faced by the world’s top general insurers a... ',
  'applications': 14,
  'jobUrl': 'https://www.reed.co.uk/jobs/senior-data-scientist/54047135',
  'city': 'London'},
 {'jobId': 53989684,
  'employerId': 501640,
  'employerName': 'Vitality',
  'emp

I convert the raw data into a pandas dataframe which will make the data easier to parse

In [40]:
df = pd.DataFrame.from_dict(extracted_data)
df

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,city
0,54047135,409522,WTW,None,None,Senior Data Scientist,London,NaN,NaN,None,30/12/2024,18/11/2024,"We are looking for a Data Scientist, with expe...",14,https://www.reed.co.uk/jobs/senior-data-scient...,London
1,53989684,501640,Vitality,None,None,Lead Data Scientist,WC2R0DT,NaN,NaN,None,06/12/2024,08/11/2024,About The Role Team – &nbsp;Data Science Worki...,29,https://www.reed.co.uk/jobs/lead-data-scientis...,London
2,54032986,543104,Jobheron,None,None,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London
3,53929241,472032,Proactive Appointments,None,None,Data Scientist,London,NaN,NaN,None,10/12/2024,29/10/2024,Data Scientist -&nbsp; Remote Working Data Sci...,245,https://www.reed.co.uk/jobs/data-scientist/539...,London
4,54054640,524441,INTEC SELECT LIMITED,None,None,Data Scientist,EC3N2EX,450.0,500.0,GBP,01/01/2025,20/11/2024,Data Scientist – 450-500pd PAYE – 7 month cont...,27,https://www.reed.co.uk/jobs/data-scientist/540...,London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,54030987,375315,Spectrum IT Recruitment,None,None,Senior PHP Software Engineer,NE11AD,50000.0,60000.0,GBP,29/11/2024,15/11/2024,We're looking for a dynamic Senior PHP Softwar...,19,https://www.reed.co.uk/jobs/senior-php-softwar...,Newcastle
9993,53905046,300264,Client Server Ltd.,None,None,Senior Software Developer C# .Net API AWS,SR12JR,70000.0,80000.0,GBP,21/11/2024,24/10/2024,Senior Software Developer / Engineer (C# .Net ...,18,https://www.reed.co.uk/jobs/senior-software-de...,Newcastle
9994,53987043,300264,Client Server Ltd.,None,None,Senior Software Developer C# .Net API AWS,SR12JR,65000.0,80000.0,GBP,06/12/2024,08/11/2024,Senior Software Developer / Engineer (C# .Net ...,12,https://www.reed.co.uk/jobs/senior-software-de...,Newcastle
9995,53980000,300264,Client Server Ltd.,None,None,Lead Backend Software Engineer C# .Net API AWS,SR12JR,70000.0,80000.0,GBP,05/12/2024,07/11/2024,Lead Backend Software Engineer / Developer (C#...,9,https://www.reed.co.uk/jobs/lead-backend-softw...,Newcastle


I will now check to see if the nested for loop executed correctly and we returned values for each city in the uk_cities list

In [41]:
df['city'].unique()

array(['London', 'Birmingham', 'Manchester', 'Glasgow', 'Leeds',
       'Liverpool', 'Bristol', 'Sheffield', 'Edinburgh', 'Leicester',
       'Coventry', 'Nottingham', 'Hull', 'Stoke-on-Trent', 'Derby',
       'Southampton', 'Reading', 'Luton', 'Wolverhampton', 'Belfast',
       'Cambridge', 'Brighton', 'Oxford', 'Newcastle', 'Cardiff', 'York'],
      dtype=object)

I will now save the raw data extracted from the API into a csv file so I can work on transforming the data without having to send more requests to the API

In [42]:
df.to_csv('reed_api_data.csv')

## Transform ##

Now I have the data saved in a pandas dataframe I can now start to transform the data. My first task will be to check for and remove any duplicated data as I suspect some of the searches will return similar job postings.

I will use the duplicated method to check for duplicate rows of data

In [49]:
df = pd.read_csv('reed_api_data.csv', index_col=0)
df.iloc[0]

jobId                                                           54047135
employerId                                                        409522
employerName                                                         WTW
employerProfileId                                                    NaN
employerProfileName                                                  NaN
jobTitle                                           Senior Data Scientist
locationName                                                      London
minimumSalary                                                        NaN
maximumSalary                                                        NaN
currency                                                             NaN
expirationDate                                                30/12/2024
date                                                          18/11/2024
jobDescription         We are looking for a Data Scientist, with expe...
applications                                       

In [50]:
duplciated_rows = df[df.duplicated(subset=['jobId'], keep=False)]
duplciated_rows

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,city
2,54032986,543104,Jobheron,NaN,NaN,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London
4,54054640,524441,INTEC SELECT LIMITED,NaN,NaN,Data Scientist,EC3N2EX,450.0,500.0,GBP,01/01/2025,20/11/2024,Data Scientist – 450-500pd PAYE – 7 month cont...,27,https://www.reed.co.uk/jobs/data-scientist/540...,London
6,54056851,400289,Huntress,NaN,NaN,Data Scientist,London,50.0,55.0,GBP,27/11/2024,20/11/2024,Data Scientist- London/Remote- 12 Months- 50- ...,31,https://www.reed.co.uk/jobs/data-scientist/540...,London
8,54064151,429392,Searchability,NaN,NaN,Data Scientist,London,50000.0,60000.0,GBP,02/01/2025,21/11/2024,Data Scientist Opportunity within a Tech4Good ...,1,https://www.reed.co.uk/jobs/data-scientist/540...,London
12,53917216,377106,Robert Walters,NaN,NaN,Data Scientist,London,60000.0,80000.0,GBP,09/12/2024,28/10/2024,Our client is in the midst of a crucial growth...,159,https://www.reed.co.uk/jobs/data-scientist/539...,London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,54049736,300264,Client Server Ltd.,NaN,NaN,Senior C# Developer .Net API AWS - Games,SR12JR,65000.0,80000.0,GBP,17/12/2024,19/11/2024,Senior C# Developer / Software Engineer (C# .N...,8,https://www.reed.co.uk/jobs/senior-c-developer...,Newcastle
9992,54030987,375315,Spectrum IT Recruitment,NaN,NaN,Senior PHP Software Engineer,NE11AD,50000.0,60000.0,GBP,29/11/2024,15/11/2024,We're looking for a dynamic Senior PHP Softwar...,19,https://www.reed.co.uk/jobs/senior-php-softwar...,Newcastle
9993,53905046,300264,Client Server Ltd.,NaN,NaN,Senior Software Developer C# .Net API AWS,SR12JR,70000.0,80000.0,GBP,21/11/2024,24/10/2024,Senior Software Developer / Engineer (C# .Net ...,18,https://www.reed.co.uk/jobs/senior-software-de...,Newcastle
9994,53987043,300264,Client Server Ltd.,NaN,NaN,Senior Software Developer C# .Net API AWS,SR12JR,65000.0,80000.0,GBP,06/12/2024,08/11/2024,Senior Software Developer / Engineer (C# .Net ...,12,https://www.reed.co.uk/jobs/senior-software-de...,Newcastle


We can see that some jobs have been duplicated multiple times

In [51]:
duplicated_job = df[df['jobId'] == 54032986]
duplicated_job

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,city
2,54032986,543104,Jobheron,NaN,NaN,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London
2193,54032986,543104,Jobheron,NaN,NaN,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London
9097,54032986,543104,Jobheron,NaN,NaN,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London
9219,54032986,543104,Jobheron,NaN,NaN,Data Scientist,London,40000.0,55000.0,GBP,27/12/2024,15/11/2024,"A Data Scientist, who must have a PhD&nbsp; qu...",55,https://www.reed.co.uk/jobs/data-scientist/540...,London


I will now remove all duplicated jobs from the dataframe

In [52]:
df_no_duplicates = df.drop_duplicates()
print("df length:", len(df))
print("df_no_duplicates length:", len(df_no_duplicates))

df length: 9997
df_no_duplicates length: 5651


In [53]:
df_no_duplicates['date'].max()

'31/10/2024'

In [54]:
df_no_duplicates.dtypes

jobId                    int64
employerId               int64
employerName            object
employerProfileId      float64
employerProfileName    float64
jobTitle                object
locationName            object
minimumSalary          float64
maximumSalary          float64
currency                object
expirationDate          object
date                    object
jobDescription          object
applications             int64
jobUrl                  object
city                    object
dtype: object